# Plotting Ballistic Ejecta from the South Pole-Aitken Impact

## Workflow:
This script runs through all the tracers in the data file and selects the tracers that follow a ballistic trajectory. It outputs the results to a csv file which can be used for plotting.

In [1]:
#import python packages 
import tools
import numpy as np
import pandas as pd

In [2]:
#import the data file. Change the path to your file :) 
data = tools.load_ballistic_data("/data2/tmd02/SPA/v10/tracer_export_all.npz", end = 400)
# print(data['ymark'].max())

In [3]:
#get data
velocity_data = tools.get_velocity(data)
radial_distance_data = tools.get_radial_distance(data)
radial_velocity_data = tools.get_radial_velocity(data)
radial_acceleration_data = tools.get_radial_acceleration(data, radial_velocity_data)
average_time = tools.get_average_time(data)

In [4]:
h_launch = 150
tracer_idxs, time_steps = tools.BallisticCheckVec(data, h_launch)
# print(time_steps)
# print(tracer_idxs[398])
# print(time_steps.max())
# print(np.min(time_steps))
# print(np.max(time_steps))
# print(time_steps.shape)
ejection_times = data['time'][time_steps]

In [5]:
tracer_material = data['trm'][tracer_idxs]

In [6]:
R0 = 1750
g0 = 0.00162

g = g0 * R0**2 / (R0 + h_launch) **2
v_tilda = tools.get_v_tilda_vec(velocity_data, tracer_idxs, time_steps, R0 + h_launch, g)

In [7]:
v_r, longitude, latitude = tools.get_launch_vector_vec(data,velocity_data, tracer_idxs, time_steps)
longitude_magnitude, longitude_direction = longitude
latitude_magnitude, latitude_direction = latitude

In [8]:
theta_launch, height, distance, azimuth = tools.get_ballistic_vectors_vec(data, velocity_data, tracer_idxs, time_steps, h_launch, R0, g0)

/home/nb3018/SPA/tools/ballistics.py:105: RuntimeWarning: invalid value encountered in arccos
  beta = np.arccos(data['zmark'] / (data['r']+ 1e-12))[:, :-1]  # angle at the centre of the moon


In [9]:
print('height min =',height.min())
print('height max =',height.max())
print('height mean =',height.mean())
print('height median', np.median(height))

height min = -1906276.9111188329
height max = 8709362.895077338
height mean = -96.39903478598517
height median 0.00016952784608481478


In [10]:
#multiply the azimuth with -1 to get the right set up
final_latitude, final_longitude = tools.track_vec(latitude[0], longitude[0], -1 * azimuth, distance)

In [12]:
c1 = final_longitude < 0
c2 = final_longitude > 0
final_longitude[c1] = final_longitude[c1] + 180
final_longitude[c2] = final_longitude[c2] - 180
final_latitude = -1 * final_latitude 
final_longitude = -1 * final_longitude

In [13]:
def make_dataframe(tracer_idxs, ejection_times, v_r, longitude, latitude, theta_launch, height, distance, final_longitude, final_latitude, tracer_material, azimuth):
    initial_df = pd.DataFrame({'Tracer Index' : tracer_idxs, 'Ejection Time' : ejection_times, 'Resultant Velocity' : v_r, 'Launch Longitude' : longitude[0], 'Launch Latitude' : latitude[0], 'Launch Angle' : theta_launch, 'Height' : height, 'Distance' : distance, 'Landing Longitude' : final_longitude, 'Landing Latitude' : final_latitude, 'Tracer Material' : tracer_material, 'Azimuth' : azimuth})
    return initial_df

In [14]:
df = make_dataframe(tracer_idxs, ejection_times, v_r, longitude, latitude, theta_launch, height, distance, final_longitude, final_latitude, tracer_material, azimuth)
# print(df['Launch Longitude'][df['Launch Longitude'] < 150])
# print(df['Ejection Time'][df['Ejection Time'] > 0].min())
# print(df.Distance)
# print(df.Distance.describe())
# print(df[df.Distance < 0])
# # print(df['Ejection Time'].describe())
# print(df.loc[1360])
# print(df['Resultant Velocity'].describe())
# # print(df['Tracer Material'].value_counts())
# print(df['Resultant Velocity'].describe())

In [15]:
df = df[df['Resultant Velocity']< 2.38] 
df

,Tracer Index,Ejection Time,Resultant Velocity,Launch Longitude,Launch Latitude,Launch Angle,Height,Distance,Landing Longitude,Landing Latitude,Tracer Material,Azimuth
9022,9024,16.027889,2.125183,137.000553,87.811654,-59.441600,11716.794955,3157.531676,4.299945e+01,15.567339,3.0,180.0
16277,16280,16.027889,0.001607,180.000000,84.048506,-32.878331,0.000277,232.058802,1.989520e-13,-76.450797,2.0,180.0
16278,16281,16.027889,0.001560,180.000000,84.444215,-34.680573,0.000287,216.786178,1.989520e-13,-77.346539,2.0,180.0
16279,16282,16.027889,0.001533,180.000000,84.840549,-30.112970,0.000215,258.630109,1.989520e-13,-76.372884,2.0,180.0
16280,16283,16.027889,0.001554,180.000000,85.236654,-35.654798,0.000298,209.096657,1.989520e-13,-78.390736,2.0,180.0
...,...,...,...,...,...,...,...,...,...,...,...,...
635508,635511,16.027889,0.001918,180.000000,70.177211,-7.150055,0.000021,1195.754725,5.684342e-14,-31.027669,1.0,180.0
635509,635512,16.027889,0.001924,180.000000,69.681914,-10.723624,0.000047,792.063323,5.684342e-14,-43.749408,1.0,180.0
635510,635513,16.027889,0.001973,180.000000,69.186979,-5.355526,0.000012,1600.090357,5.684342e-14,-16.799308,1.0,180.0
635511,635514,16.027889,0.001973,180.000000,68.691201,-4.859748,0.000010,1764.237333,5.684342e-14,-10.929285,1.0,180.0


In [16]:
#save the dataframe as a csv file 
df.to_csv("full_tracer_v10.csv", index=False)    